In [1]:
from ultralytics import YOLO
import torch
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import requests
from bs4 import BeautifulSoup
import difflib
from collections import defaultdict
from itertools import cycle
from flask import Flask, render_template, request
from werkzeug.utils import secure_filename
import os

app = Flask(__name__)

@app.route('/')
def home():
    print("Current working directory:", os.getcwd())
    print("Files in templates:", os.listdir('templates'))
    return render_template('home.html')

@app.route('/', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return 'No file part'
    file = request.files['file']

    if file.filename == '':
        return 'No selected file'
    
    filename = secure_filename(file.filename)
    file.save(os.path.join("/Users/marcelosouza/Documents/MBA USP/5. TCC/Insulin Suggester/Codes/flask/uploads", filename))

    # Here is where you would call your main function to process the image
    # I have replaced it with a placeholder function
    processed_image_path = main(filename)
    
    return render_template('result.html', image_path=processed_image_path)

def predict_yolov8(path_checkpoint, best_pt, path_img):
    """
    Uses the pre-trained YOLOv8 model to predict a new picture.

    Args:
        path_checkpoint (str): Path of the checkpoint for the pre-trained YOLOv8 for a instance segmentation task.
        best_pt (str): Name of the best checkpoint file.
        path_img (str): Path of the predicted picture.

    Returns:
        results (list): A list containing all nutritional info extracted from the website.
    """  
    model = YOLO(path_checkpoint+best_pt)  # load a pretrained model (recommended for training)
    device = torch.device("mps")
    model.to(device)
    results = model(path_img)
    return results

def detected_labels(results_yolo):
    """
    Extracts the labels from the YOLOv8 results.

    Args:
        results (list): A list containing all nutritional info extracted from the website.

    Returns:
        detected_labels (list): A list containing all YOLOv8 detected labels.
    """  
    results_object = results_yolo[0]
    detected_classes = results_object.boxes.cls
    detected_labels = [results_object.names[int(i)] for i in detected_classes]
    return detected_labels

def search_food(detected_labels):
    """
    Searches the given food at the nutrional website, returning the corresponding urls.

    Args:
        detected_labels (list): A list containing all YOLOv8 detected labels.

    Returns:
        food_urls (list): A list containing all urls given the food names.
    """  
    food_urls = {}
    for food in detected_labels:
        search_url = "https://www.fatsecret.com/calories-nutrition/search?q={}".format(food)
        response = requests.get(search_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        search_results = soup.find('table', {'class': 'generic searchResult'}).find_all('tr')
        for result in search_results:
            link = result.find('a')
            if link:
                food_urls[food] = "https://www.fatsecret.com" + link['href']
                break  # We only need the first result
    return food_urls

def get_nutritional_info(url):
    """
    Extracts the nutritional info from a given website using beautifulsoup4.

    Args:
        url (str): Website URL that gives the nutritional info.

    Returns:
        nutrition_info (list): A list containing all nutritional info extracted from the website.
    """  
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Get food name
    food_name = soup.find('h1', {'style': 'text-transform:none'}).text.strip()

    nutrition_info = {food_name: {}}

    # Get serving size
    serving_size = soup.find('td', {'class': 'serving_size black us serving_size_value'}).text.strip()
    nutrition_info[food_name]['Serving Size'] = serving_size

    nutrition_facts = soup.find('div', {'class': 'nutrition_facts us'})
    if nutrition_facts:
        # Get calories
        calories_label = nutrition_facts.find('div', {'class': 'hero_label black'})
        calories_value = nutrition_facts.find('div', {'class': 'hero_value black right'})
        if calories_label and calories_value:
            nutrition_info[food_name][calories_label.text.strip()] = calories_value.text.strip()

        # Get Total Fat, Protein, and Total Carbohydrate
        nutrient_labels = nutrition_facts.find_all('div', {'class': 'nutrient black left'})
        for label in nutrient_labels:
            nutrient_value = label.find_next('div', {'class': 'nutrient value left'})
            if nutrient_value and label.text.strip():
                nutrition_info[food_name][label.text.strip()] = nutrient_value.text.strip()

        # Get vitamins and minerals
        nutrient_labels = nutrition_facts.find_all('div', {'class': 'nutrient left'}, recursive=False)
        for label in nutrient_labels:
            nutrient_value = label.find_next('div', {'class': 'nutrient value left'})
            if nutrient_value and label.text.strip() and nutrient_value.text.strip() != "0":
                nutrition_info[food_name][label.text.strip()] = nutrient_value.text.strip()

    return nutrition_info

def add_qualification_info(nutrition_info, thresholds):
    """
    Compares the nutrition values with a list of thresholds in order to have a qualitative outcome to show to the customer.
    **************** still ongoing, not finished ****************

    Args:
        nutrition_info (list): A list containing all nutritional info extracted from the website.
        thresholds (list): A list containing suggested values for nutrition parameters.

    Returns:
        qualifications (list): A list of the qualitative version of the nutritional values.
    """ 
    qualifications = []

    if float(nutrition_info['Protein'][:-1]) > thresholds['protein']:
        qualifications.append("Good Amount of Protein")

    if float(nutrition_info['Calories'][:-1]) > thresholds['calories']:
        qualifications.append("High Amount of Calories")

    return qualifications

def find_best_match(label, nutrition_keys):
    """
    Compares the nutrition label names with the YOLOv8 predicted labels.

    Args:
        label (list): A list containing the labels predicted by YOLOv8.
        nutrition_keys (list): A list containing the labels found on the website.

    Returns:
        best_match (list): A list of the matching labels.
    """  
    best_match_ratio = 0
    best_match = ""
    
    for key in nutrition_keys:
        sm = difflib.SequenceMatcher(None, label.lower(), key.lower())
        match_ratio = sm.ratio()
        
        if match_ratio > best_match_ratio:
            best_match_ratio = match_ratio
            best_match = key
            
    return best_match

def LabeledPicture(path_img, results, nutrition_info):
    """
    Aggregates nutritional info on the predicted picture, together with the boundary boxes.

    Args:
        path_img (str): Path of the predicted picture.
        results (list): A list containing all predicted results from YOLOv8.
        nutrition_info (list): A list containing all nutritional info extracted from the website.

    Returns:
        plt (Figure): A matplotlib figure of the food plate with the boundary boxes and nutritional info.
    """    
    results_object = results[0]

    # Load image
    img = cv2.imread(path_img)

    # Convert BGR image to RGB for matplotlib
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Create figure and axes
    fig, ax = plt.subplots(1)

    # Display the image
    ax.imshow(img_rgb)

    # Define color dict
    colors = cycle(['red', 'green', 'blue', 'magenta', 'cyan'])

    # Find the highest and lowest y-coordinates among all bounding boxes
    y_values = [box.xyxy[0][1].item() for box in results_object.boxes]
    highest_y = round(max(y_values))
    lowest_y = round(min(y_values))

    # Create a dictionary to map each class_id to a color
    color_dict = defaultdict(lambda: next(colors))

    # For each detected object
    for box in results_object.boxes:
        class_id = results_object.names[box.cls[0].item()]
        cords = box.xyxy[0].tolist()
        cords = [round(x) for x in cords]
        conf = round(box.conf[0].item() * 100)

        # Create a Rectangle patch
        rect = patches.Rectangle((cords[0], cords[1]), cords[2]-cords[0], cords[3]-cords[1], linewidth=1, edgecolor=color_dict[class_id], facecolor='none')

        # Add the patch to the Axes
        ax.add_patch(rect)

        # Find the best match for class_id in the nutrition_keys
        best_match = find_best_match(class_id, list(nutrition_info.keys()))

        # Prepare text
        text = f"{best_match} ({nutrition_info[best_match]['Serving Size']}) [P: {conf}%]\n"
        text += f"{nutrition_info[best_match]['Calories']} Cal | Fat: {nutrition_info[best_match]['Total Fat']} | Carb: {nutrition_info[best_match]['Total Carbohydrate']} | Prot: {nutrition_info[best_match]['Protein']}\n"

        # Add vitamins and minerals if they exist and are not 0
        vitamins_minerals = [key for key in nutrition_info[best_match].keys() if key not in ['Serving Size', 'Calories', 'Total Fat', 'Total Carbohydrate', 'Protein', 'Cholesterol', 'Sodium'] and nutrition_info[best_match][key] not in ['0', '-']]
        if vitamins_minerals:
            # Aggregate Vitamins
            vitamins = [vit for vit in vitamins_minerals if 'Vitamin' in vit]
            vitamins_minerals = [vit for vit in vitamins_minerals if 'Vitamin' not in vit]
            if vitamins:
                vitamins = [vit.replace("Vitamin ", "") for vit in vitamins]
                vitamins_minerals.append('Vitamin ' + ', '.join(vitamins))
            text += ', '.join(vitamins_minerals)
        
        # Determine y-position for the label
        if cords[1] == highest_y:
            y_position = cords[1] + 70
        elif cords[1] == lowest_y:
            y_position = cords[1] - 70
        else:
            y_position = cords[1]
        
        # Place the label
        plt.text(x=cords[0], y=y_position, s=text, color='white', verticalalignment='top',
                 bbox={'color': color_dict[class_id], 'pad': 0}, fontsize=6)

    plt.axis('off')
    return plt

def main():
    """
    Main function that orchestrates the entire program execution.

    The function should include all necessary steps to predict a picture in a pre-trained YOLOv8 model (instance segmentation task),
    extract the labels and coordinates, send this info to beautifulsoup4, which takes those labels and use in a website to
    extract nutritional info. Finally, the picture is shown, together with the boundary boxes and the nutritional info.
    """
    # thresholds = {
    #     'protein': 10,
    #     'calories': 200
    # }

    path_checkpoint = "/Users/marcelosouza/Documents/MBA USP/5. TCC/Insulin Suggester/Codes/Iterations_YOLOv8/20230412_yolov8s-seg_30epochs/runs/segment/train/weights/"
    best_yolo_model = 'best.pt'
    img_path = "/Users/marcelosouza/Documents/MBA USP/5. TCC/Insulin Suggester/Datasets/schema_food.png"
    
    results = predict_yolov8(path_checkpoint, best_yolo_model, img_path)
    food_names = detected_labels(results)
    
    # Search for each food name
    food_urls = search_food(food_names)
    
    nutrition_info = {}
    for food, url in food_urls.items():
        nutrition_info.update(get_nutritional_info(url))
        
    img = LabeledPicture(img_path, results, nutrition_info)
    img.show()

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.129.5:5001
Press CTRL+C to quit
127.0.0.1 - - [02/Jun/2023 00:02:39] "GET / HTTP/1.1" 200 -


Current working directory: /Users/marcelosouza/Documents/MBA USP/5. TCC/Insulin Suggester/Codes/flask
Files in templates: ['home.html', 'result.html']


127.0.0.1 - - [02/Jun/2023 00:02:39] "GET /favicon.ico HTTP/1.1" 404 -
192.168.129.5 - - [02/Jun/2023 00:03:05] "GET / HTTP/1.1" 200 -


Current working directory: /Users/marcelosouza/Documents/MBA USP/5. TCC/Insulin Suggester/Codes/flask
Files in templates: ['home.html', 'result.html']
